In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [43.6 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchp

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Challenge16").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [4]:
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

In [6]:
vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RMDCHWD0Y5OZ9|          3|            0|          1|   N|                N|
| RZSL0BALIYUNU|          5|            0|          0|   N|                Y|
| RIZR67JKUDBI0|          3|            0|          1|   N|                Y|
|R27HL570VNL85F|          5|            0|          0|   N|                Y|
|R34EBU9QDWJ1GD|          5|            0|          0|   N|                Y|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|                N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|                Y|
|R3GYQ5W8JHP8SB|          5|            0|          0|   N|                Y|
|R30SHYQXGG5EYC|          5|            0|          0|   N|                Y|
|R14YLXA56NP51I|          5|            1|          1|   N|     

In [7]:
vine_df.printSchema()


root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [8]:
vine_gt20 = vine_df.filter("total_votes>=20")

In [9]:
vine_gt20.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|  RMRD6SMF2AUQ|          3|            4|         21|   N|                N|
| RMPCXKWX3T57Y|          1|            1|         73|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|     

In [10]:
vine_gt50pct = vine_gt20.filter("helpful_votes/total_votes>=.50")

In [11]:
vine_gt50pct.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
|R2R6JPF9KOD2HJ|          5|           19|         20|   N|                Y|
|R2J0ZZGFXKM8KR|          2|           21|         22|   N|     

In [14]:
vine_y = vine_gt50pct.filter("vine=='Y'")

In [15]:
vine_y.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1R9RU7JW0MFR2|          4|           20|         23|   Y|                N|
|R19EFYNN3W8Q07|          5|           26|         32|   Y|                N|
|R34DJ1R8AEU0SG|          5|           29|         35|   Y|                N|
|R25P5CXK5L9RHF|          5|          146|        161|   Y|                N|
|R2E9VZB3I4LSN5|          5|           55|         59|   Y|                N|
| RKYLHZL7EPELX|          4|           19|         25|   Y|                N|
|R1U13EKGQD3ZE6|          5|           22|         25|   Y|                N|
| RYW05F1MUEF01|          5|           87|        102|   Y|                N|
|R2SW4NXNO7HZJ5|          4|           28|         33|   Y|                N|
|R2016NFLSUR97Y|          2|           26|         37|   Y|     

In [16]:
vine_n = vine_gt50pct.filter("vine=='N'")

In [17]:
vine_n.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
|R2R6JPF9KOD2HJ|          5|           19|         20|   N|                Y|
|R2J0ZZGFXKM8KR|          2|           21|         22|   N|     

In [38]:
vine_gt50pct.groupBy('star_rating').sum().orderBy("star_rating").show()

+-----------+----------------+------------------+----------------+
|star_rating|sum(star_rating)|sum(helpful_votes)|sum(total_votes)|
+-----------+----------------+------------------+----------------+
|          1|            1532|             53248|           64261|
|          2|            1510|             28719|           33662|
|          3|            3903|             57449|           64883|
|          4|           10812|            135453|          145302|
|          5|           41230|            415228|          444491|
+-----------+----------------+------------------+----------------+



In [39]:
# Sum
vine_gt50pct.agg({'star_rating':'sum'}).show()

+----------------+
|sum(star_rating)|
+----------------+
|           58987|
+----------------+



In [37]:
vine_y.groupBy('star_rating').sum().orderBy("star_rating").show()

+-----------+----------------+------------------+----------------+
|star_rating|sum(star_rating)|sum(helpful_votes)|sum(total_votes)|
+-----------+----------------+------------------+----------------+
|          2|               2|                26|              37|
|          3|              27|               409|             483|
|          4|              64|               437|             493|
|          5|             170|              1541|            1741|
+-----------+----------------+------------------+----------------+



In [40]:
# Sum
vine_y.agg({'star_rating':'sum'}).show()

+----------------+
|sum(star_rating)|
+----------------+
|             263|
+----------------+



In [36]:
vine_n.groupBy('star_rating').sum().orderBy("star_rating").show()

+-----------+----------------+------------------+----------------+
|star_rating|sum(star_rating)|sum(helpful_votes)|sum(total_votes)|
+-----------+----------------+------------------+----------------+
|          1|            1532|             53248|           64261|
|          2|            1508|             28693|           33625|
|          3|            3876|             57040|           64400|
|          4|           10748|            135016|          144809|
|          5|           41060|            413687|          442750|
+-----------+----------------+------------------+----------------+



In [41]:
vine_n.agg({'star_rating':'sum'}).show()

+----------------+
|sum(star_rating)|
+----------------+
|           58724|
+----------------+



In [66]:
vine_n_pct = vine_n.groupBy('star_rating').sum().withColumnRenamed('sum(star_rating)','starsum')
  
      

In [67]:
vine_n_pct.show()

+-----------+-------+------------------+----------------+
|star_rating|starsum|sum(helpful_votes)|sum(total_votes)|
+-----------+-------+------------------+----------------+
|          1|   1532|             53248|           64261|
|          3|   3876|             57040|           64400|
|          5|  41060|            413687|          442750|
|          4|  10748|            135016|          144809|
|          2|   1508|             28693|           33625|
+-----------+-------+------------------+----------------+



In [73]:
import pyspark.sql.functions as f

from pyspark.sql.window import Window

vine_n_pct = vine_n_pct.withColumn('percent',f.col('starsum')/f.sum('starsum').over(Window.partitionBy())*100)

vine_n_pct.orderBy("star_rating").show()


+-----------+-------+------------------+----------------+------------------+
|star_rating|starsum|sum(helpful_votes)|sum(total_votes)|           percent|
+-----------+-------+------------------+----------------+------------------+
|          1|   1532|             53248|           64261|2.6088141134800082|
|          2|   1508|             28693|           33625| 2.567944962877188|
|          3|   3876|             57040|           64400| 6.600367822355426|
|          4|  10748|            135016|          144809|18.302567944962878|
|          5|  41060|            413687|          442750|  69.9203051563245|
+-----------+-------+------------------+----------------+------------------+



In [81]:
vine_y_pct = vine_y.groupBy('star_rating').sum().withColumnRenamed('sum(star_rating)','starsum')

In [82]:
vine_y_pct.show()

+-----------+-------+------------------+----------------+
|star_rating|starsum|sum(helpful_votes)|sum(total_votes)|
+-----------+-------+------------------+----------------+
|          3|     27|               409|             483|
|          5|    170|              1541|            1741|
|          4|     64|               437|             493|
|          2|      2|                26|              37|
+-----------+-------+------------------+----------------+



In [83]:
vine_y_pct = vine_y_pct.withColumn('percent',f.col('starsum')/f.sum('starsum').over(Window.partitionBy())*100)
vine_y_pct.orderBy("star_rating").show()

+-----------+-------+------------------+----------------+------------------+
|star_rating|starsum|sum(helpful_votes)|sum(total_votes)|           percent|
+-----------+-------+------------------+----------------+------------------+
|          2|      2|                26|              37|0.7604562737642585|
|          3|     27|               409|             483| 10.26615969581749|
|          4|     64|               437|             493|24.334600760456272|
|          5|    170|              1541|            1741| 64.63878326996198|
+-----------+-------+------------------+----------------+------------------+

